In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException, NoSuchElementException
from bs4 import BeautifulSoup
import requests
import re
from collections import defaultdict
import time



In [2]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
 
from sql_init import Review, Base, Category
 
engine = create_engine('sqlite:///reviews.db')
# Bind the engine to the metadata of the Base class so that the
# declaratives can be accessed through a DBSession instance
Base.metadata.bind = engine
 
DBSession = sessionmaker(bind=engine)
# A DBSession() instance establishes all conversations with the database
# and represents a "staging zone" for all the objects loaded into the
# database session object. Any change made against the objects in the
# session won't be persisted into the database until you call
# session.commit(). If you're not happy about the changes, you can
# revert all of them back to the last commit by calling
# session.rollback()
session = DBSession()

In [3]:
def click(element, n=5):
    for _ in range(n):
        try:
            element.click()
            break
        except WebDriverException:
            continue
    raise WebDriverException('Couldnt click {} parents up'.format(n))
            
    

In [4]:
driver = webdriver.Chrome(executable_path="./chromedriver")
driver.implicitly_wait(3)
driver.get("https://www.amazon.com/s/ref=sr_nr_p_89_0?fst=as%3Aoff&rh=i%3Aaps%2Ck%3Aadidas%2Cp_89%3Aadidas&keywords=adidas&ie=UTF8&qid=1507481929&rnid=2528832011")

In [5]:
main_window = driver.current_window_handle

In [6]:
for i in driver.find_elements_by_class_name("a-expander-prompt"):
    i.find_element_by_xpath('..').click()

In [7]:
e = []
for i in driver.find_elements_by_class_name("s-ref-text-link"):
    if i.text == "Get It by Tomorrow":
        break
    e.append(i)

In [10]:
[x.text for x in e]

["Boys' Fashion",
 "Boys' Sneakers",
 "Boys' Clothing",
 "Boys' Skateboarding Shoes",
 "Boys' Running Shoes",
 "Men's Fashion",
 "Men's Fashion Sneakers",
 "Men's Running Shoes",
 "Men's Clothing",
 "Men's Road Running Shoes",
 "Men's Cross-Training Shoes",
 "Men's Skateboarding Shoes",
 "Men's Athletic Shoes",
 "Men's Activewear",
 "Men's Walking Shoes",
 "Men's Tennis & Racquet Sport Shoes",
 "Women's Fashion",
 "Women's Fashion Sneakers",
 "Women's Running Shoes",
 "Women's Clothing",
 "Women's Activewear",
 "Women's Road Running Shoes",
 "Women's Skateboard Shoes",
 "Women's Cross Training Shoes",
 "Women's Walking Shoes",
 "Women's Basketball Shoes",
 "Women's Tennis & Racquet Sport Shoes",
 "Girls' Fashion",
 "Girls' Sneakers",
 "Girls' Skateboarding Shoes",
 'Sports & Fitness',
 "Men's Exercise & Fitness Apparel",
 "Women's Soccer Clothing",
 "Men's Soccer Clothing",
 "Women's Exercise & Fitness Apparel",
 'Sports Fan Shop',
 'Sports Fan Clothing',
 'Baby Clothing & Shoes',
 "Ba

In [24]:
e = e[0]

In [25]:
def check_category(e):
    categories = session.query(Category).filter_by(category_name=e.text).all()
    if len(categories) == 0:
        new_category = Category(category_name=e.text,completed=False)
        session.add(new_category)
        session.commit()
        return False
    elif len(categories) >= 1:
        return categories[0].completed

    
    
            

In [26]:
categories

NameError: name 'categories' is not defined

In [27]:
get_category_reviews(e[0])

NameError: name 'get_category_reviews' is not defined

In [30]:
e[1].send_keys(Keys.CONTROL + Keys.SHIFT + Keys.RETURN)

In [11]:
for handle in driver.window_handles:
    if handle != main_window:
        driver.switch_to_window(handle)
        
#     print ("Handle = ",handle)
#     driver.switch_to_window(handle);
#     elem = driver.find_element_by_tag_name("title")
#     print (elem.get_attribute("text"))

In [46]:
driver.switch_to_window(driver.window_handles[1])

In [48]:
driver.window_handles

['CDwindow-579a6ad3-3e31-45c8-87d7-5256ed239123']

In [12]:
links = []
elements = []
for i in driver.find_elements_by_class_name("s-access-title"):
    links.append(i.find_element_by_xpath('..').get_attribute('href'))
    elements.append(i)

In [13]:
elements[0].click()
time.sleep(3)

In [14]:
a_s = driver.find_elements_by_tag_name('a')
#TODO Error handling for when there are no reviews
for e in a_s:
    if re.search('See all [0-9,.]* customer reviews', e.text):
        e.click()
        break

In [44]:
e = driver.find_element_by_class_name('a-last')

In [45]:
try:
    e.find_element_by_tag_name('a').get_attribute('href')
except NoSuchElementException:
    

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"tag name","selector":"a"}
  (Session info: chrome=62.0.3202.94)
  (Driver info: chromedriver=2.33.506106 (8a06c39c4582fbfbab6966dbb1c38a9173bfb1a2),platform=Mac OS X 10.12.6 x86_64)


In [40]:
e

'Next→'

In [ ]:
def check_if_last():
    

In [ ]:
try:
    e = e.find_element_by_tag_name('a')
except NoSuchElementException:
    driver.close()
    

In [ ]:
driver.switch_to_window(main_window)

In [ ]:
e.text

In [ ]:
e.click()

In [ ]:
reviews = defaultdict(dict)
# for e in driver.find_elements_by_xpath('*//i[@data-hook="review-star-rating"]'):
def scrape_review(e):
    star_rating = e.find_element_by_class_name('a-icon-alt').get_attribute('innerHTML')[0]
    e = e.find_element_by_xpath('../../../..')
#     review_id = e.get_attribute('id')
#     reviews[review_id]['star_rating'] = star_rating
#     reviews[review_id]['review_title'] = e.find_element_by_class_name('review-title').text
#     reviews[review_id]['review_date'] = e.find_element_by_class_name('review-date').text[3:]
#     reviews[review_id]['text'] = e.find_element_by_class_name('review-text').text
    
    new_review = Review(review_id = e.get_attribute('id'),
    star_rating = int(star_rating),
    review_title = e.find_element_by_class_name('review-title').text,
    review_date = e.find_element_by_class_name('review-date').text[3:],
    text = e.find_element_by_class_name('review-text').text)
    session.add(new_review)
    session.commit()
    
    
    
    
    
    
    
    

In [15]:
def scrape_review(e, session):
    star_rating = e.find_element_by_class_name('a-icon-alt').get_attribute('innerHTML')[0]
    e = e.find_element_by_xpath('../../../..')
    new_review = Review(review_id = e.get_attribute('id'),
    star_rating = int(star_rating),
    review_title = e.find_element_by_class_name('review-title').text,
    review_date = e.find_element_by_class_name('review-date').text[3:],
    text = e.find_element_by_class_name('review-text').text)
    session.add(new_review)
    session.commit()

In [ ]:
old_category = Category(category_name='old category',completed=False)
session.add(old_category)
session.commit()

In [ ]:
session.query(Category).all()[0].completed

In [ ]:
for e in driver.find_elements_by_xpath('*//i[@data-hook="review-star-rating"]'):
    e.parent.click()
    break

In [91]:
session.rollback()

In [92]:
s = session.query(Review).all()

In [95]:
[x.star_rating for x in s]

[5, 5, 5, 5, 5, 5, 5, 4, 1, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5]

In [ ]:
[print(i.category_name) for i in s]

In [77]:
driver = webdriver.Chrome(executable_path="./chromedriver")
driver.implicitly_wait(3)
url = 'https://www.amazon.com/s/gp/search/ref=sr_nr_p_89_0?fst=as%3Aoff&rh=i%3Aaps%2Ck%3Aadidas%2Cp_89%3Aadidas&keywords=adidas&ie=UTF8&qid=1511382355&rnid=2528832011'
driver.get(url)
main_window = driver.current_window_handle


In [78]:
for i in driver.find_elements_by_class_name("a-expander-prompt"):
    i.find_element_by_xpath('..').click()


In [79]:
# Iterate through the categories
category = driver.find_elements_by_class_name("s-ref-text-link")[0]

In [80]:
driver.switch_to_window(driver.window_handles[0])

In [81]:
if not check_category(category, session):
    # Open new window for category
    category.send_keys(Keys.CONTROL + Keys.SHIFT + Keys.RETURN)
    # Switch to new window 
    

InvalidRequestError: This Session's transaction has been rolled back due to a previous exception during flush. To begin a new transaction with this Session, first issue Session.rollback(). Original exception was: (sqlite3.IntegrityError) UNIQUE constraint failed: reviews.review_id [SQL: 'INSERT INTO reviews (review_id, star_rating, review_title, review_date, text) VALUES (?, ?, ?, ?, ?)'] [parameters: ('RL17959WYMDQO', 5, 'for sizing', 'July 23, 2015', "I wear a size 8 in women's so I ordered a size 6 instead of 7 (the normal women's to kids conversion) bc I know these run large and they fit perfectly! Just order two sizes smaller than the women's size you would regularly get and they'll be perfect!")]

In [ ]:
driver.switch_to_window(driver.window_handles[1])
product = driver.find_elements_by_class_name("s-access-title")[0]
product.click()


In [ ]:
# driver.switch_to_window(driver.window_handles[2])
a_s = driver.find_elements_by_tag_name('a')
for e in a_s:
    if re.search('See all [0-9,.]* customer reviews', e.text):
        e.send_keys(Keys.CONTROL + Keys.SHIFT + Keys.RETURN)
        driver.switch_to_window(driver.window_handles[2])
        break


In [87]:
def check_category(e, session):
    categories = session.query(Category).filter_by(category_name=e.text).all()
    if len(categories) == 0:
        new_category = Category(category_name=e.text,completed=False)
        session.add(new_category)
        session.commit()
        return False
    elif len(categories) >= 1:
        return categories[0].completed

def scrape_review(e, session):


    star_rating = e.find_element_by_class_name('a-icon-alt').get_attribute('innerHTML')[0]
    e = e.find_element_by_xpath('../../../..')
    if session.query(Review).filter_by(review_id=e.id).all():
        pass    
    new_review = Review(review_id = e.get_attribute('id'),
    star_rating = int(star_rating),
    review_title = e.find_element_by_class_name('review-title').text,
    review_date = e.find_element_by_class_name('review-date').text[3:],
    text = e.find_element_by_class_name('review-text').text)
    session.add(new_review)
    session.commit()
    return

In [88]:
session.rollback()

In [86]:
review.

<selenium.webdriver.remote.webelement.WebElement (session="dff6ce2e207b35ab1df1a77f3ea010bb", element="0.3214756417273712-1")>

In [69]:
if len(session.query(Review).filter_by(review_id='RL17959WYMDQO').all()) > 0:
    print ('blah')

blah


In [89]:
review=driver.find_elements_by_xpath('*//i[@data-hook="review-star-rating"]')[0]
scrape_review(review, session)


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: reviews.review_id [SQL: 'INSERT INTO reviews (review_id, star_rating, review_title, review_date, text) VALUES (?, ?, ?, ?, ?)'] [parameters: ('R2YARFSOTSPD1W', 5, 'reading other reviews helped me pick the right size!1', 'July 22, 2015', "Just got these shoes in the mail and was a bit nervous to see if I had ordered the right size given that most of the reviews stated that the shoe run ... (222 characters truncated) ... iewers' advice and drop down a size and I'm glad that I did. I hope this helps you choose the right size!\nThey are super cute and I love the colors!")]

In [ ]:

    try:
        e.find_element_by_tag_name('a').click()
    except NoSuchElementException:
        cat = session.query(Category).filter_by(category_name=e.text).first()
        cat.completed = True
        session.commit()
        driver.close()
        driver.switch_to_window(driver.window_handles[1])
        break